In [18]:
import pandas as pd 
# CSV file path (using raw string)
csv_file = r"C:\Users\mahmoud mohmed\Desktop\GUI RH\project.csv"

data = pd.read_csv(csv_file)
row = data.head(1000) 


In [2]:
import csv
import pymongo
# Replace with your MongoDB connection string if different from localhost
mongo_uri = "mongodb://localhost:27017/"

# CSV file path (using raw string)
csv_file = r"C:\Users\mahmoud mohmed\Desktop\GUI RH\project.csv"

# Connect to MongoDB
client = pymongo.MongoClient(mongo_uri)

# Access the database (replace "projectBigData" if needed)
db = client["projectBigData"]

# Create collections
patient_collection = db["Patient"]
health_data_collection = db["Health_Data"]
diseases_collection = db["disease"]
diabetes_collection = db["diabetes"]
arthritis_collection = db["arthritis"]
doctor_collection = db["Doctors"]
address_collection = db["Address"]
reader=[]
# Read data from CSV file with error handling
with open(csv_file, 'r', encoding='utf-8', errors='replace') as csvfile:
    reader = csv.DictReader(csvfile)
    rows = [row for row in reader]
for row_num, row in enumerate(rows[:100]):
        # Extract doctor data
        doctor_name = row["Doctor name"]
        
        # Check if doctor already exists in the collection
        doctor = doctor_collection.find_one({"Name": doctor_name})
        if not doctor:
            # If doctor doesn't exist, insert into doctor collection
            doctor_id = doctor_collection.insert_one({"Name": doctor_name}).inserted_id
        else:
            doctor_id = doctor["_id"]

        # Prepare address data
        address_data = {
            "City": row["City"],
            "Street": row["Street"],
            "Country": row["Country"]
        }
        # Insert address data and get the inserted ID
        address_id = address_collection.insert_one(address_data).inserted_id

        # Prepare patient data with doctor and address reference
        patient_data = {
            "Sex": row["Sex"],
            "Age_Category": (row['Age_Category']),
            "Height_(cm)": float(row['Height_(cm)']),
            "Weight_(kg)": float(row['Weight_(kg)']),
            "Smoking_History": row['Smoking_History'],
            "BMI":float(row['BMI']),
            "DoctorID": doctor_id,  # Reference to doctor document
            "Addresses": address_id
        }

        # Insert patient data
        patient_id = patient_collection.insert_one(patient_data).inserted_id

        # Prepare health record
        health_record = {
            "General_Health": row["General_Health"],  
            "Checkup": row["Checkup"],
            "Exercise": row["Exercise"],
            "Green_Vegetables_Consumption": row["Green_Vegetables_Consumption"],
            "Fruit_Consumption": row["Fruit_Consumption"],
            "Alcohol_Consumption": row["Alcohol_Consumption"],
            "PatientID": patient_id  # Reference the patient ID
        }
        # Insert health record
        health_id = health_data_collection.insert_one(health_record).inserted_id

        # Prepare diseases data
        diseases_data = {
            "Heart_Disease": row["Heart_Disease"],
            "Skin_Cancer": row["Skin_Cancer"],
            "Other_Cancer": row["Other_Cancer"],
            "Depression": row["Depression"],
            "PatientID": patient_id, 
            "HealthDataID": health_id 
        }

        # Insert the diseases dataa
        diseases_collection.insert_one(diseases_data)

        # Prepare diabetes data
        diabetes_record = {
            "HasDiabetes": row["Diabetes"],
            "HealthDataID": health_id  # Reference Health Data ID
        }
        # Insert the diabetes data
        diabetes_collection.insert_one(diabetes_record)

        # Prepare arthritis data
        arthritis_record = {
            "HasArthritis": row["Arthritis"],
            "HealthDataID": health_id  # Reference Health Data ID
        }
        # Insert the arthritis data
        arthritis_collection.insert_one(arthritis_record)

print("Data stored in the MongoDB database.")


Data stored in the MongoDB database.


In [12]:
type(doctor_name)

str

In [13]:
doctor_name

'Kirsti'

In [20]:
row

,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,...,BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption,Doctor name,Street,Country,City
0,Poor,Within the past 2 years,No,No,No,No,No,No,Yes,Female,...,14.54,Yes,0,30,16,12,Ancell,8568 Ridgeway Avenue,China,Houbai
1,Very Good,Within the past year,No,Yes,No,No,No,Yes,No,Female,...,28.29,No,0,30,0,4,Redford,763 Schmedeman Court,China,Jiedu
2,Very Good,Within the past year,Yes,No,No,No,No,Yes,No,Female,...,33.47,No,4,12,3,16,Laureen,79 Bartelt Plaza,United States,Shawnee Mission
3,Poor,Within the past year,Yes,Yes,No,No,No,Yes,No,Male,...,28.73,No,0,30,30,8,Abel,429 Norway Maple Parkway,Brazil,Rio Bonito
4,Good,Within the past year,No,No,No,No,No,No,No,Male,...,24.37,Yes,0,8,4,0,Stefanie,354 Hollow Ridge Parkway,Vietnam,SÆ¡n HÃ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Excellent,Within the past year,Yes,No,No,No,No,No,No,Female,...,26.58,Yes,0,30,5,0,Roth,525 Clyde Gallagher Avenue,Switzerland,Bern
996,Very Good,Within the past year,No,No,No,No,Yes,No,No,Male,...,31.75,No,0,30,30,12,Abel,5 Dixon Terrace,Canada,Notre-Dame-des-Prairies
997,Good,Within the past 5 years,Yes,No,No,No,No,No,No,Male,...,28.75,Yes,12,20,16,2,Melicent,6922 Pawling Road,Poland,MurÃ³w
998,Very Good,Within the past year,Yes,No,No,No,No,No,No,Male,...,21.91,No,7,30,0,20,Mala,68963 Burrows Junction,Indonesia,Kalapagada


In [52]:
# Find patients who are smoker "Yess" and have a BMI greater than 40
result = patient_collection.find({"Smoking_History":"Yes", "BMI": {"$gt": "40"}})
for doc in result:
    print(doc)

{'_id': ObjectId('663b7cb72a82ccf10fb6abfd'), 'Sex': 'Female', 'Age_Category': '60-64', 'Height_(cm)': '163', 'Weight_(kg)': '120.2', 'Smoking_History': 'Yes', 'BMI': '45.49', 'DoctorID': ObjectId('663b7cb72a82ccf10fb6abfb'), 'Addresses': [{'City': 'Banbalah', 'Street': '102 Mitchell Court', 'Country': 'Tunisia', '_id': ObjectId('663b7cb72a82ccf10fb6abfc')}]}
{'_id': ObjectId('663b7cb72a82ccf10fb6adcb'), 'Sex': 'Female', 'Age_Category': '65-69', 'Height_(cm)': '168', 'Weight_(kg)': '115.67', 'Smoking_History': 'Yes', 'BMI': '41.16', 'DoctorID': ObjectId('663b7cb72a82ccf10fb6adc9'), 'Addresses': [{'City': 'Pondokdalem', 'Street': '2626 Garrison Terrace', 'Country': 'Indonesia', '_id': ObjectId('663b7cb72a82ccf10fb6adca')}]}
{'_id': ObjectId('663b7cb72a82ccf10fb6b1cd'), 'Sex': 'Female', 'Age_Category': '60-64', 'Height_(cm)': '160', 'Weight_(kg)': '107.95', 'Smoking_History': 'Yes', 'BMI': '42.16', 'DoctorID': ObjectId('663b7cb72a82ccf10fb6b1cb'), 'Addresses': [{'City': 'Bukui', 'Street'

In [54]:
# Find patients who have diabetes
result = patient_collection.find({"Addresses": {"$exists": False}})
for doc in result:
    print(doc)


In [56]:
# Find patients who have diabetes
result = diabetes_collection.find({"HasDiabetes": "Yes"})
for doc in result:
    print(doc)


{'_id': ObjectId('663b7cb72a82ccf10fb6aab7'), 'HasDiabetes': 'Yes', 'HealthDataID': ObjectId('663b7cb72a82ccf10fb6aab5')}
{'_id': ObjectId('663b7cb72a82ccf10fb6aabe'), 'HasDiabetes': 'Yes', 'HealthDataID': ObjectId('663b7cb72a82ccf10fb6aabc')}
{'_id': ObjectId('663b7cb72a82ccf10fb6aac5'), 'HasDiabetes': 'Yes', 'HealthDataID': ObjectId('663b7cb72a82ccf10fb6aac3')}
{'_id': ObjectId('663b7cb72a82ccf10fb6aaef'), 'HasDiabetes': 'Yes', 'HealthDataID': ObjectId('663b7cb72a82ccf10fb6aaed')}
{'_id': ObjectId('663b7cb72a82ccf10fb6aafd'), 'HasDiabetes': 'Yes', 'HealthDataID': ObjectId('663b7cb72a82ccf10fb6aafb')}
{'_id': ObjectId('663b7cb72a82ccf10fb6ab19'), 'HasDiabetes': 'Yes', 'HealthDataID': ObjectId('663b7cb72a82ccf10fb6ab17')}
{'_id': ObjectId('663b7cb72a82ccf10fb6ab2e'), 'HasDiabetes': 'Yes', 'HealthDataID': ObjectId('663b7cb72a82ccf10fb6ab2c')}
{'_id': ObjectId('663b7cb72a82ccf10fb6ab35'), 'HasDiabetes': 'Yes', 'HealthDataID': ObjectId('663b7cb72a82ccf10fb6ab33')}
{'_id': ObjectId('663b7c

In [57]:
# Find patients sorted by BMI in descending order
result = patient_collection.find().sort("BMI", pymongo.DESCENDING)
for doc in result:
    print(doc)


{'_id': ObjectId('663b7cb82a82ccf10fb6bc44'), 'Sex': 'Female', 'Age_Category': '60-64', 'Height_(cm)': '168', 'Weight_(kg)': '145.6', 'Smoking_History': 'No', 'BMI': '51.81', 'DoctorID': ObjectId('663b7cb82a82ccf10fb6bc42'), 'Addresses': [{'City': 'Koynare', 'Street': '8 Melvin Terrace', 'Country': 'Bulgaria', '_id': ObjectId('663b7cb82a82ccf10fb6bc43')}]}
{'_id': ObjectId('663b7cb82a82ccf10fb6b97f'), 'Sex': 'Female', 'Age_Category': '60-64', 'Height_(cm)': '175', 'Weight_(kg)': '158.76', 'Smoking_History': 'No', 'BMI': '51.69', 'DoctorID': ObjectId('663b7cb82a82ccf10fb6b97d'), 'Addresses': [{'City': 'ChabaÅ™ovice', 'Street': '5511 Westport Circle', 'Country': 'Czech Republic', '_id': ObjectId('663b7cb82a82ccf10fb6b97e')}]}
{'_id': ObjectId('663b7cb72a82ccf10fb6b213'), 'Sex': 'Female', 'Age_Category': '50-54', 'Height_(cm)': '160', 'Weight_(kg)': '131.09', 'Smoking_History': 'No', 'BMI': '51.19', 'DoctorID': ObjectId('663b7cb72a82ccf10fb6b211'), 'Addresses': [{'City': 'Ciudad Guayana',

In [61]:
result = patient_collection.insert_one({"BMI": "30", "Smoking_History": "No", "Weight_(kg)": "70", "Height_(cm)": "170", "Age_Category": "35-70", "Sex": "Male"})



Inserted ID: 663b81892a82ccf10fb6c5ce


In [70]:
from bson import ObjectId
patient_id_to_update = ObjectId("663b81892a82ccf10fb6c5ce")
patient_collection.update_one({"_id": patient_id_to_update}, {"$set": {"Smoking_History": "Yes"}})


UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [86]:
# Create an index on the "DoctorID" field in the "Patient" collection
patient_collection.create_index("DoctorID",pymongo.DESCENDING)


AttributeError: 'int' object has no attribute 'in_transaction'

In [3]:
result = disease_collection.find({"$match": {"Heart_Disease": "Yes", "Depression": "Yes"}},
	{"$count": "total"})
for doc in result:
    print(doc)

NameError: name 'disease_collection' is not defined

In [6]:
pipeline = [
    { '$match': { 'gender': 'female', 'depression': 'yes' } },
    { '$count': 'number_of_females_with_depression' }
]

result = collection.aggregate(pipeline)
    for doc in result:
        print(doc)

IndentationError: unexpected indent (1137484752.py, line 7)

In [7]:
res= health_data_collection.find({
    "Heart_Disease": "Yes",
    "Alcohol_Consumption": "0"           

})

In [8]:
for doc in res:
    print(doc)

In [17]:
patient_collection.drop_index("BMI_-1_Age_Category_1")

In [19]:
res = patient_collection.list_indexes()
for r in res:
    print (r)

SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('BMI', -1), ('Age_Category', 1)])), ('name', 'BMI_-1_Age_Category_1')])


In [4]:

map_function = """
function() {
    emit(this.Age_Category, parseFloat(this.BMI));
}
"""

# Define the reduce function
reduce_function = """
function(key, values) {
    return Array.sum(values) / values.length;
}
"""

# Perform MapReduce
res = db.command({
    "mapReduce": "Patient",
    "map": map_function,
    "reduce": reduce_function,
    "out": "Output_sum"
})


SyntaxError: incomplete input (619404578.py, line 21)